# Create a regional or country-wide crop type map

## Background

Once we are satisfied with the map results for the test areas, we can proceed to apply the model over a larger region or an entire country. To limit the memory use, we will break a large area into tiles, make predictions over tiles and merge the results.

## Description

This notebook can be used to generate a crop type map over a region defined in a shapefile.

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [1]:
import xarray as xr
from joblib import load
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import json
import pickle
import subprocess

import datacube
from datacube.utils import geometry
from datacube.utils.cog import write_cog
from odc.io.cgroups import get_cpu_quota

from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.classification import predict_xr
from deafrica_tools.spatial import xr_rasterize

from feature_collection import feature_layers

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pa

## Create Dask cluster for running predictions

We use dask to parallel and speed up the processing.

In [3]:
ncpus = round(get_cpu_quota())
print("ncpus = " + str(ncpus))

# client = create_local_dask_cluster(return_client=True, n_workers=1, threads_per_worker=ncpus)
create_local_dask_cluster()

ncpus = 4


/usr/local/lib/python3.8/dist-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45055 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:33993 Dashboard: /user/mseehaber@frontiersi.com.au/proxy/45055/status,Cluster Workers: 1 Cores: 4 Memory: 28.14 GB


## Load model and class labels

We use the model trained and saved in the [fit classifier notebook](3_Fit_classifier.ipynb). it is important that the list of features used match the model.

In [4]:
model_basepath = "Results/Model/"

# Choose model and load
model_path = os.path.join(model_basepath, "rf_removecorrfeaturesgt0p9_simplified_cv.joblib")
model = load(model_path).set_params(n_jobs=1)


# Get label dictionary
labels_path = os.path.join(model_basepath, "class_labels.json")
with open(labels_path, "r") as json_file:
    labels_dict = json.load(json_file)

# Get model features
feautres_path = os.path.join(model_basepath, "rf_removecorrfeaturesgt0p9_simplified_features.json")
with open(feautres_path, "r") as json_file:
    features_dict = json.load(json_file)
    
features = features_dict["features"]

In [5]:
len(features)

46

## Load area of interest (AOI)

Two sets of tiles that cover the country of Mozambique are provided as shapefiles in the `Data` folder. 
The smaller ones are suitable for processing with 32 GB of memory.

In [6]:
output_crs="EPSG:6933"

In [7]:
# Choose the test area shapefile
tiles_file = "Data/Mozambique_tiles_smaller.shp"
tiles_gdf = gpd.read_file(tiles_file).to_crs(output_crs)

# Set results path
output_folder = "Results/Map"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


For testing, we select a level 2 (district) admin region in Mozambique to generate the crop type map.

In [8]:
# Pick a district name
district_name = "Matsinho 2" #"Lichinga"

# Set basename for the output files
if district_name is None:
    output_basename = "Mozambique"
    boundary_file = "Data/Mozambique_boundary.shp"
else:
    output_basename = f"{district_name.replace(' ','_')}"

> To run this workflow over a level 2 admin region, you will need to change `GID_2` and `NAME_2`. 
>
> **Note:** If running over a level 2 admin region, you will also need to specify a `district_name` in the previous step which corresponds the to `NAME_2` values 

In [9]:
if district_name is not None:
    mozambique_admins = gpd.read_file("Data/Mozambique_admin_gadm_level3.gpkg")
    mozambique_level3 = mozambique_admins.dissolve(by="GID_3").to_crs(output_crs)
    admin = mozambique_level3[mozambique_level3["NAME_3"]==district_name]
    boundary_file = f"Results/Map/{district_name.replace(' ','_')}_boundary.geojson"
    admin.to_file(boundary_file)

In [10]:
# Visulize the location

if district_name is not None:
    admin.explore(
        tiles="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        attr='Imagery @2022 Landsat/Copernicus, Map data @2022 Google',
        popup=True,
        cmap='viridis',
        style_kwds=dict(color='red', fillOpacity=0, weight=3),
    )

### Select tiles that overlap the admin region 

In [11]:
if district_name is not None:
    AOIs_gdf = tiles_gdf[tiles_gdf.overlaps(admin.iloc[0].geometry)]
else:
    AOIs_gdf = tiles_gdf

In [12]:
# check number and size of tiles

print("# of tiles:", len(AOIs_gdf))
print("Area of each tile (sq km):", int(AOIs_gdf.area.mean()/1e6))

# of tiles: 2
Area of each tile (sq km): 900


In [13]:
# visulize location of the tiles

AOIs_gdf.explore(
    tiles = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}", 
    attr ='Imagery @2022 Landsat/Copernicus, Map data @2022 Google',
    popup=True,
    cmap='viridis',
    style_kwds=dict(color= 'red', fillOpacity= 0, weight= 3),
    )

## Create the query for running the predictions

We use the query saved from the training data collection notebook to ensure data from the same periods are retrieved. However, only selected features will be used. 

> We add `dask_chunks` to the query parameter so the data will be lazy-loaded and only the features used by the model will be calculated.

In [14]:
# Load the query used for fitting
query_file = os.path.join(model_basepath, "query.pickle")

with open(query_file, "rb") as f:
    query = pickle.load(f)
    
# Specify any specific additions to the data query -- e.g. dask_chunks for enabling parallel computation
if ncpus<=4:
    dask_chunks = {"x": 1500, "y": 1500}
else:
    dask_chunks = {"x": 4000, "y": 4000}
query.update({"dask_chunks": dask_chunks})

query

{'semiannual_geomad_times': {'semiannual_2021_07': '2021-07-01',
  'semiannual_2022_01': '2022-01-01'},
 'monthly_ndvi_time_range': ('2021-10', '2022-09'),
 'ls_fc_cover_times': {'Q4_2021': slice('2021-10-01', '2021-12-31', None),
  'Q1_2022': slice('2022-01-01', '2022-03-31', None),
  'Q2_2022': slice('2022-04-01', '2022-06-30', None),
  'Q3_2022': slice('2022-07-01', '2022-09-30', None)},
 'resolution': (-10, 10),
 'output_crs': 'EPSG:6933',
 'dask_chunks': {'x': 1500, 'y': 1500}}

## Apply classification model to the AOI

The model will be applied over each tile, producing a prediction map and a probabilities map. The maps are saved as Cloud-Optimized Geotiffs (COGs).

> Tiles are processed in sequence. For each tile, the processing needs to fit into the compute resources available in the sandbox. Make the tile size smaller if you run out of memory. For production of a map over a large region or country, consider applying for [a large sandbox (with more CPUs and momery)](
https://helpdesk.digitalearthafrica.org/portal/en/community/topic/call-for-application-for-access-to-large-sandboxes-15-processing-cores-and-120-gb-of-memory)

If output files for a tile already exist, processing for the tile can be skipped. This is useful if the process fails partway through, or if you are logged out of the sandbox before all tiles are completed.

In [15]:
skip_exisiting = True

In [16]:
%%time

dc = datacube.Datacube(app="crop_type_ml")

for index in range(0, len(AOIs_gdf)):
    aoi = AOIs_gdf.iloc[index]
    print(f"Processing Polygon {index}")

    # Check if polygon has already been processed. If so, skip
    output_filename = os.path.join(
        output_folder, f"{output_basename}_tile{index:03d}_croptype_prediction.tif")
    probabilities_filename = os.path.join(
        output_folder, f"{output_basename}_tile{index:03d}_croptype_probabilities.tif")
    if skip_exisiting and os.path.exists(output_filename) and os.path.exists(probabilities_filename):
        print("Completed; Skipping")
        continue

    # set up query based on aoi geometry
    geom = geometry.Geometry(geom=aoi.geometry, crs=AOIs_gdf.crs)
    query.update({"geopolygon": geom})

    # Load the feature data
    print("    Loading feature data")
    data = feature_layers(query)

    # Only keep features that are used by the model
    data = data[features]

#     #predict using the imported model
    predicted = predict_xr(model,
                           data,
                           proba=True,
                           persist=False,
                           clean=True,
                           return_input=False
                           ).compute().astype(np.uint8)

    # Mask with crop extent
    crop_mask_query = dict((k, query[k]) for k in (
        'resolution', 'output_crs', 'geopolygon'))
    crop_mask_query.update({"time": "2019"})

    # Load the crop mask
    print("    Loading crop_mask")
    # "filtered" crop mask is used by default, alternatively, unfiltered "mask" measurement can be used
    crop_mask_measurement = 'filtered'
    crop_mask = dc.load(product="crop_mask", measurements=[crop_mask_measurement], **crop_mask_query)

    # Create a mask for the aoi
    print("    Getting AOI mask")
    aoi_mask = xr_rasterize(
        gdf=gpd.GeoDataFrame({"Polygon": [index], "geometry": [
                             aoi.geometry]}, crs=AOIs_gdf.crs),
        da=predicted,
        transform=predicted.geobox.transform,
        crs=output_crs,
    )

    # set the no data value
    NODATA = 255

    # Mask the predictions to
    print("    Preparing predictions")
    predicted_masked = (
        predicted.Predictions.where(
            (crop_mask[crop_mask_measurement] == 1) & (aoi_mask == 1), NODATA)
    )

    predicted_masked.attrs["nodata"] = NODATA

    # Write to cog
    print(f"    Writing predictions to {output_filename}")
    write_cog(
        predicted_masked,
        fname=output_filename,
        overwrite=True,
        nodata=255,
    )

    del predicted_masked

    probability_masked = (
        predicted.Probabilities.where(
            (crop_mask[crop_mask_measurement] == 1) & (aoi_mask == 1), NODATA)
    )

    probability_masked.attrs["nodata"] = NODATA

    print(f"    Writing probabilities to {probabilities_filename}")
    write_cog(
        probability_masked,
        fname=probabilities_filename,
        overwrite=True,
        nodata=255,
    )

    del probability_masked

    del crop_mask
    del aoi_mask

Processing Polygon 0
    Loading feature data
predicting...
   probabilities...
    Loading crop_mask
    Getting AOI mask
    Preparing predictions
    Writing predictions to Results/Map/Matsinho_2_tile000_croptype_prediction.tif
    Writing probabilities to Results/Map/Matsinho_2_tile000_croptype_probabilities.tif
Processing Polygon 1
    Loading feature data
predicting...
   probabilities...
    Loading crop_mask
    Getting AOI mask
    Preparing predictions
    Writing predictions to Results/Map/Matsinho_2_tile001_croptype_prediction.tif
    Writing probabilities to Results/Map/Matsinho_2_tile001_croptype_probabilities.tif
CPU times: user 2min 10s, sys: 20.7 s, total: 2min 31s
Wall time: 12min 23s


## Mosaic maps

The tiled maps are merged and clipped to the admin boundaries.

In [17]:
out_mosaic = f"{output_folder}/{output_basename}_merged_croptype_prediction.tif"
# remove the merged file if it already exists
if os.path.exists(out_mosaic): 
    subprocess.run(f"rm {out_mosaic}", shell=True)
    
merge_cmd = f"gdal_merge.py -o {out_mosaic} -co COMPRESS=Deflate -ot Byte {output_folder}/{output_basename}_tile???_croptype_prediction.tif -init 255 -a_nodata 255"
subprocess.run(merge_cmd, shell=True)

out_mosaic = f"{output_folder}/{output_basename}_merged_croptype_probabilities.tif"
# remove the merged file if it already exists
if os.path.exists(out_mosaic): 
    subprocess.run(f"rm {out_mosaic}", shell=True)

merge_cmd = f"gdal_merge.py -o {out_mosaic} -co COMPRESS=Deflate -ot Byte {output_folder}/{output_basename}_tile???_croptype_probabilities.tif -init 255 -a_nodata 255"
subprocess.run(merge_cmd, shell=True)


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args='gdal_merge.py -o Results/Map/Matsinho_2_merged_croptype_probabilities.tif -co COMPRESS=Deflate -ot Byte Results/Map/Matsinho_2_tile???_croptype_probabilities.tif -init 255 -a_nodata 255', returncode=0)

> The merged and clipped images can be used for further analysis in the Sandbox or downloaded to be inspected and analyzed in a different GIS software.

In [18]:
input_mosaic=f'{output_folder}/{output_basename}_merged_croptype_prediction.tif'
out_mosaic=f'{output_folder}/{output_basename}_croptype_prediction.tif'
# remove the merged file if it already exists
if os.path.exists(out_mosaic): 
    subprocess.run(f"rm {out_mosaic}", shell=True)

gdal_cmd=f"gdalwarp -of GTiff -cutline {boundary_file} -crop_to_cutline -co COMPRESS=DEFLATE {input_mosaic} {out_mosaic}"
subprocess.run(gdal_cmd, shell=True)

input_mosaic=f'{output_folder}/{output_basename}_merged_croptype_probabilities.tif'
out_mosaic=f'{output_folder}/{output_basename}_croptype_probabilities.tif'
# remove the merged file if it already exists
if os.path.exists(out_mosaic): 
    subprocess.run(f"rm {out_mosaic}", shell=True)

gdal_cmd=f"gdalwarp -of GTiff -cutline {boundary_file} -crop_to_cutline -co COMPRESS=DEFLATE {input_mosaic} {out_mosaic}"
subprocess.run(gdal_cmd, shell=True)

Creating output file that is 2178P x 3438L.
Processing Results/Map/Matsinho_2_merged_croptype_prediction.tif [1/1] : 0Using internal nodata values (e.g. 255) for image Results/Map/Matsinho_2_merged_croptype_prediction.tif.
Copying nodata values from source Results/Map/Matsinho_2_merged_croptype_prediction.tif to destination Results/Map/Matsinho_2_croptype_prediction.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2178P x 3438L.
Processing Results/Map/Matsinho_2_merged_croptype_probabilities.tif [1/1] : 0Using internal nodata values (e.g. 255) for image Results/Map/Matsinho_2_merged_croptype_probabilities.tif.
Copying nodata values from source Results/Map/Matsinho_2_merged_croptype_probabilities.tif to destination Results/Map/Matsinho_2_croptype_probabilities.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args='gdalwarp -of GTiff -cutline Results/Map/Matsinho_2_boundary.geojson -crop_to_cutline -co COMPRESS=DEFLATE Results/Map/Matsinho_2_merged_croptype_probabilities.tif Results/Map/Matsinho_2_croptype_probabilities.tif', returncode=0)